<hr /> 

## Machine Learning Marathon parte 1

```
Desafio inspirado em trabalho para disciplina de Aprendizagem de Máquina do curso de Especialização em Inteligência Artificial Aplicada da Universidade Federal do Paraná (UFPR) 2021
```

<hr /> 


### Objetivo do desafio

Para os datasets:

- Volume;
- Biomassa;
- Admissão;
- Alunos Ensino Médio;

Treinar os seguintes modelos de **regressão**:

- Rede Neural Artificial (com Hold out, cross validation e grid search)
- K-Nearest Neighbors
- Support Vector Machine (Com hold out, cross validation e grid search)
- Random Forest (com hold out, cross validation e grid search)

Apresentando em uma tabela, todos os modelos elencando as seguintes métricas de regressão:

- Parâmetros do modelo;
- R2
- Syx
- Correlação de Pearson
- RMSE
- MAE

<hr />

 Observações:
 
- A tabela de modelos para cada dataset deve ser ordenada por maior R2;
- Apresentar um gráfico de residuos para cada tabela para o modelo de maior R2;

<hr /> 


In [1]:
# Modulos e bibliotecas necessários

# Visualização
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# modelos
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

# métricas de regressão
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from scipy.stats import pearsonr
import numpy as np

# Opta por utilziar a visualização linda do seaborn <3
plt.style.use('seaborn')

In [2]:
# Define os caminhos para os arquivos de dados em um dicionário
files = {
    'volume': 'datasets/volume/Material 02 - 3 – Estimativa de Volume - Dados.csv',
    'biomassa': 'datasets/biomass/Material 02 - 4 - R - Biomassa - Dados.csv',
    'admissao': 'datasets/admissao/Material 02 - 8 – R - Admissao - Dados.csv',
    'alunos_em': 'datasets/alunos_em/Material 02 - 10 – Alunos - Dados.csv'
}

# Funções de métricas manualmente implementadas
def r2(predicted, target):
    return 1 - (sum((predicted - target)**2) / sum((predicted - np.mean(target)**2)))

# Syx
def syx(predicted, target, feature_size):
    return  sum((predicted - target)**2) / (len(target) - feature_size)


<hr /> 

# Dataset 1 - Volume

<hr /> 


In [3]:
volume_df = pd.read_csv(files['volume'])
volume_df.head()

,PROJETO,CLONE,Idade,Hdom,AreaBasal,Volume
0,Fazenda Velha,C3,21.5,8.8,3.2,12.1
1,Fazenda Velha,A3,21.9,10.2,3.9,66.8
2,Fazenda Velha,A3,26.1,10.1,4.2,72.1
3,Fazenda Velha,A4,22.0,10.0,4.2,35.9
4,Parada,B7,23.4,15.3,4.3,44.5


## Criação da partição de dados
<br />
Os dados serão divididos em 80% para treino e 20% para teste.

- Desconsiderou-se as variáveis categóricas `PROJETO` e `CLONE`;

In [4]:
volume_X_train, volume_X_test, volume_y_train, volume_y_test = train_test_split(
    volume_df[['Idade', 'Hdom', 'AreaBasal']].values,
    volume_df.Volume,
    test_size=0.2,
    random_state=2154
)

# Exibe os shapes das partições
print('Train X shape ', volume_X_train.shape)
print('Train y shape ', volume_y_train.shape)
print('Test X shape ', volume_X_test.shape)
print('Test y shape ', volume_y_test.shape)

Train X shape  (244, 3)
Train y shape  (244,)
Test X shape  (62, 3)
Test y shape  (62,)


## Rede Neural Artificial
<br />
 A rede neural utilizada será uma *Multi Layer Perceptron Regressor (MLP REGRESSOR)* do sci-kit learn
 
 A rede será treinada com os métodos:
 - *Hold out*
 - *Cross Validation*
 - *Grid Search*

In [5]:
%%capture

# Inicializa uma coleção de RNAs a se preencher
volume_mlps = {
    'mlp_hold_out': [],
    'mlp_cross_val': [],
    'mlp_grid_search': []
}

# MLP hold out
volume_mlp = MLPRegressor(random_state=2154)
volume_mlp.fit(volume_X_train, volume_y_train)

# MLP com cros val
volume_cv_mlp = GridSearchCV(volume_mlp, param_grid={}, cv=10)
volume_cv_mlp.fit(volume_X_train, volume_y_train)

# MLP com grid
parameter_space = {
    'hidden_layer_sizes': [(1, 5, 10), (50,50,50), (50,100,50), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}
volume_grid_mlp = GridSearchCV(volume_mlp, parameter_space, n_jobs=-1, cv=10)
volume_grid_mlp.fit(volume_X_train, volume_y_train)

mlps = [volume_mlp, volume_cv_mlp, volume_grid_mlp]

# Métricas
for key, model in zip(volume_mlps.keys(), mlps):
    # Parametros
    predicts = model.predict(volume_X_test)
    # R2 Manual
    volume_mlps[key].append(r2(predicts, volume_y_test))
    # R2 do sklearn
    volume_mlps[key].append(r2_score(predicts, volume_y_test))
    # SYX
    volume_mlps[key].append(syx(predicts, volume_y_test, len(volume_X_test[0])))
    # Pearson
    volume_mlps[key].append(pearsonr(volume_y_test, predicts)[0])
    # RMSE
    volume_mlps[key].append(np.sqrt(mean_squared_error(volume_y_test, predicts)))
    # MAE
    volume_mlps[key].append(mean_absolute_error(volume_y_test, predicts))


# parametros dos modelos
volume_mlps['mlp_hold_out'].insert(0, {
    'size': volume_mlp.get_params()['hidden_layer_sizes'],
    'alpha': volume_mlp.get_params()['alpha'],
})
volume_mlps['mlp_cross_val'].insert(0, {
    'size': volume_cv_mlp.estimator.get_params()['hidden_layer_sizes'],
    'alpha': volume_cv_mlp.estimator.get_params()['alpha'],
})
volume_mlps['mlp_grid_search'].insert(0, {
    'size': volume_grid_mlp.best_params_['hidden_layer_sizes'],
    'alpha': volume_grid_mlp.best_params_['alpha'],
})

## K-Nearest Neighbors
<br />

Treinar o KNN com:
- *Grid Search*


In [6]:
%%capture

# Treina a KNN
volume_knn = KNeighborsRegressor()
volume_knn = GridSearchCV(volume_knn, {'n_neighbors': [1, 3, 5, 7, 9]}, n_jobs=-1, cv=10)
volume_knn.fit(volume_X_train, volume_y_train)

# predicts
knn_predicts = volume_knn.predict(volume_X_test)

knn_metrics = {
    'knn': [
        {'k': volume_knn.best_params_['n_neighbors']},
        r2(knn_predicts, volume_y_test),  # R2 Manual
        r2_score(knn_predicts, volume_y_test),  # R2 do sklearn
        syx(knn_predicts, volume_y_test, len(volume_X_test[0])),  # SYX
        pearsonr(volume_y_test, knn_predicts)[0],  # Pearson
        np.sqrt(mean_squared_error(volume_y_test, knn_predicts)), # RMSE
        mean_absolute_error(volume_y_test, knn_predicts)  # MAE
    ]
}

## Support Vector Machine -  SVR
<br />
 A SVM utilizada será a *Support Vector Regressor* disponibilizada pelo SKlearn
 
 A SVR será treinada com os métodos:
 - *Hold out*
 - *Cross Validation*
 - *Grid Search*

In [7]:
%%capture

# Inicializa uma coleção de SVMs a se preencher
volume_svms = {
    'svm_hold_out': [],
    'svm_cross_val': [],
    'svm_grid_search': []
}

# SVR hold out
volume_svr = SVR()
volume_svr.fit(volume_X_train, volume_y_train)

# SVR com cross val
volume_cv_svr = GridSearchCV(volume_svr, param_grid={}, cv=10)
volume_cv_svr.fit(volume_X_train, volume_y_train)

# SVR com grid
param_grid = [
    {
        'C': [2.0, 2.1, 2.2, 2.5, 2.7, 3.0, 4.0, 5.0, 50],
        'degree': [-1, 1, 1.5, 2, 3, 4, 5],
        'epsilon': [0.1, 0.2, 0.3, 0.5, 0.7]
    },
]
volume_grid_svr = GridSearchCV(volume_svr, param_grid, n_jobs=-1, cv=10)
volume_grid_svr.fit(volume_X_train, volume_y_train)


svrs = [volume_svr, volume_cv_svr, volume_grid_svr]

# Métricas
for key, model in zip(volume_svms.keys(), svrs):
    predicts = model.predict(volume_X_test)
    # R2 Manual
    volume_svms[key].append(r2(predicts, volume_y_test))
    # R2 do sklearn
    volume_svms[key].append(r2_score(predicts, volume_y_test))
    # SYX
    volume_svms[key].append(syx(predicts, volume_y_test, len(volume_X_test[0])))
    # Pearson
    volume_svms[key].append(pearsonr(volume_y_test, predicts)[0])
    # RMSE
    volume_svms[key].append(np.sqrt(mean_squared_error(volume_y_test, predicts)))
    # MAE
    volume_svms[key].append(mean_absolute_error(volume_y_test, predicts))


# Parametros dos modelos
volume_svms['svm_hold_out'].insert(0, {
    'C': volume_svr.get_params()['C'],
    'epsilon': volume_svr.get_params()['epsilon'],
})
volume_svms['svm_cross_val'].insert(0, {
    'C': volume_cv_svr.estimator.get_params()['C'],
    'epsilon': volume_cv_svr.estimator.get_params()['epsilon'],
})
volume_svms['svm_grid_search'].insert(0, {
    'C': volume_grid_svr.best_params_['C'],
    'epsilon': volume_grid_svr.best_params_['epsilon'],
})

## Random Forest
<br />
 A Floresta Aleatória utilizada será a *Random Forest Regressor* disponibilizada pelo SKlearn
 
 A RF será treinada com os métodos:
 - *Hold out*
 - *Cross Validation*
 - *Grid Search*

In [10]:
%%capture

# Inicializa uma coleção de RF a se preencher
volume_rfs = {
    'rf_hold_out': [],
    'rf_cross_val': [],
    'rf_grid_search': []
}

# RF hold out
volume_rf = RandomForestRegressor()
volume_rf.fit(volume_X_train, volume_y_train)

# RF com cross val
volume_cv_rf = GridSearchCV(volume_rf, param_grid={}, cv=10)
volume_cv_rf.fit(volume_X_train, volume_y_train)

param_grid = [
    {
        'n_estimators': [10, 50, 100, 200, 500],
        'max_depth': [1, 2, 3, 4, 5],
    },
]
volume_grid_rf = GridSearchCV(volume_rf, param_grid, n_jobs=-1, cv=10)
volume_grid_rf.fit(volume_X_train, volume_y_train)

rfs = [volume_rf, volume_cv_rf, volume_grid_rf]

# Métricas
for key, predicts in zip(volume_rfs.keys(), rfs):
    predicts = model.predict(volume_X_test)
    # R2 Manual
    volume_rfs[key].append(r2(predicts, volume_y_test))
    # R2 do sklearn
    volume_rfs[key].append(r2_score(predicts, volume_y_test))
    # SYX
    volume_rfs[key].append(syx(predicts, volume_y_test, len(volume_X_test[0])))
    # Pearson
    volume_rfs[key].append(pearsonr(volume_y_test, predicts)[0])
    # RMSE
    volume_rfs[key].append(np.sqrt(mean_squared_error(volume_y_test, predicts)))
    # MAE
    volume_rfs[key].append(mean_absolute_error(volume_y_test, predicts))

# Parametros dos modelos
volume_rfs['rf_hold_out'].insert(0, {
    'n_estimators': volume_rf.get_params()['n_estimators'],
    'max_depth': volume_rf.get_params()['max_depth'],
})
volume_rfs['rf_cross_val'].insert(0, {
    'n_estimators': volume_cv_rf.estimator.get_params()['n_estimators'],
    'max_depth': volume_cv_rf.estimator.get_params()['max_depth'],
})
volume_rfs['rf_grid_search'].insert(0, {
    'n_estimators': volume_grid_rf.best_params_['n_estimators'],
    'max_depth': volume_grid_rf.best_params_['max_depth'],
})

## Agrupando a tabela de modelos para o dataset Volumes

In [11]:
volume_models = pd.DataFrame()
volume_collections = [volume_mlps, volume_svms, volume_rfs, knn_metrics]

for collection in volume_collections:
    volume_models = volume_models.append(pd.DataFrame.from_dict(collection).T)

volume_models.columns = ['Params', 'R2 Manual', 'R2 Sklearn', 'SYX', 'Pearson', 'RMSE', 'MAE']
volume_models.sort_values(by=['R2 Manual'], ascending=False)

,Params,R2 Manual,R2 Sklearn,SYX,Pearson,RMSE,MAE
svm_hold_out,"{'C': 1.0, 'epsilon': 0.1}",1.153781,-12.514327,21882.144318,0.827616,144.302914,120.166135
svm_cross_val,"{'C': 1.0, 'epsilon': 0.1}",1.153781,-12.514327,21882.144318,0.827616,144.302914,120.166135
svm_grid_search,"{'C': 50, 'epsilon': 0.7}",1.082198,0.589168,11693.0762,0.845593,105.485934,82.047433
rf_hold_out,"{'n_estimators': 100, 'max_depth': None}",1.082198,0.589168,11693.0762,0.845593,105.485934,82.047433
rf_cross_val,"{'n_estimators': 100, 'max_depth': None}",1.082198,0.589168,11693.0762,0.845593,105.485934,82.047433
rf_grid_search,"{'n_estimators': 200, 'max_depth': 4}",1.082198,0.589168,11693.0762,0.845593,105.485934,82.047433
mlp_hold_out,"{'size': (100,), 'alpha': 0.0001}",1.070423,0.500642,10019.346045,0.846014,97.644964,75.697968
mlp_cross_val,"{'size': (100,), 'alpha': 0.0001}",1.070423,0.500642,10019.346045,0.846014,97.644964,75.697968
mlp_grid_search,"{'size': (100,), 'alpha': 0.05}",1.070294,0.501599,10001.102586,0.846675,97.556026,75.692644
knn,{'k': 7},1.06526,0.670183,9284.351038,0.868762,93.995257,75.491475


<hr />

# Dataset 2 - Biomassa

<hr />